In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision
import cv2
import glob

In [ ]:
def run_yolo(model,image_path):
    """
    ids = {
        CIH__id: 0,
        Cell: 4,
        face_id: 1,
    }
    """
    
    results = model(image_path)

    
    results.save()
    
    return image, output

In [ ]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath


# To run the model You just need to clone yolov5 repository from github 
# and add new_yolo_2.pt into yolov5\models\ directory
# https://github.com/ultralytics/yolov5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.hub.load(r"yolov5", 'custom', path=r"yolov5\models\new_yolo_2.pt", source='local') 

#model.conf = 0.50  # confidence threshold (0-1)
#model.iou = 0.45  # NMS IoU threshold (0-1)  

model = model.to(device)

In [ ]:
# Class Names: ["Cell Phone", "Face", "Hand", "OOD", "Cell Phone Attached"]

results = {}
ids = {
    "CIH__id": 0,
    "Cell": 4,
    "face_id": 1,
}

status = ["0","1"]
labels,inputs = [], []

for i,s in enumerate(status):
    for index,image_path in enumerate(glob.glob("BLURRED_IMAGES\\"+s+"\\*")):
        """
        PART 2: FILL THIS AREA
        """


In [ ]:
def run_yolo_hand_Extraction(model,image_path,ids):
    """
    ids = {
        CIH__id: 0,
        Cell: 4,
        face_id: 1,
    }
    """
    
    results = model(image_path)

    """
    PART 3: FILL THIS AREA
    """
        
    return 0


status = ["1","0"]
labels,inputs = [], []

ids = {
    "CIH__id": 0,
    "Cell": 4,
    "face_id": 1,
}

for i,s in enumerate(status):
    for index,image_path in enumerate(glob.glob("BLURRED_IMAGES\\"+s+"\\*")):
        """
        PART 3: FILL THIS AREA
        """
